In [1]:
import pandas as pd
import json


with open("../out/data.json") as fout:
    raw_data = json.load(fout)

user_template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:

{}

"""

system_template = """### Response:

{} 

"""

end_template = """{}### End"""

agg_conversations = []
for idx_thread, thread in enumerate(raw_data):
    conversations = ""
    prev_owner = ""
    
    for idx_post in range(len(thread)):
        post_curr = thread[idx_post]
        if post_curr["role"] == "user":
            if prev_owner == "user":
                conversations += "\n" + post_curr["text"]
            elif prev_owner == "system":
                conversations += "\n" + user_template.format(post_curr["text"])
            else:
                conversations += "\n" + user_template.format(post_curr["text"])

        if post_curr["role"] == "system":
            if prev_owner == "system":
                conversations += "\n" + post_curr["text"]
            elif prev_owner == "user":
                conversations += "\n" + system_template.format(post_curr["text"])
            else:
                conversations += "\n" + user_template.format(post_curr["text"])

        prev_owner = post_curr["role"]
    agg_conversations.append(end_template.format(conversations))

# create dataframe
test_conv_dataframe = pd.DataFrame(agg_conversations, columns=["conversations"])
test_conv_dataframe

,conversations
0,\nBelow is an instruction that describes a tas...
1,\nBelow is an instruction that describes a tas...
2,\nBelow is an instruction that describes a tas...
3,\nBelow is an instruction that describes a tas...
4,\nBelow is an instruction that describes a tas...
...,...
1406,\nBelow is an instruction that describes a tas...
1407,\nBelow is an instruction that describes a tas...
1408,\nBelow is an instruction that describes a tas...
1409,\nBelow is an instruction that describes a tas...


In [2]:
test_conv_dataframe.to_csv("../data/test-conversations.csv", sep="\t", index=None)